## Globus

In [33]:
from tinydb import TinyDB
globus_db = TinyDB("../../globus.json")
len(globus_db.all())

3557

In [34]:
globus_places = [record for record in globus_db.all() if "place" in record]
globus_places[0]

{'name': 'Абрамовщина 3-я',
 'url': 'https://globustut.by/abramovsch/index.htm',
 'rating': 3,
 'photo_count': 4,
 'place': {'name': 'Абрамовщина 3-я',
  'location': 'Сморгонский район, Гродненская область',
  'coords': [54.60153, 26.53344],
  'sights': [{'name': 'оборонительные сооружения 1-й мировой войны',
    'image': 'https://orda.of.by/.ga/a/abramovsch/oboron_main/nf/abramovschina_3-ya-kuhnya_1mir-06.05.06-d-179.jpg',
    'props': [['Дата строительства', ['1915—18 гг.']],
     ['Рейтинг (1..4)', ['увидеть по дороге']],
     ['Координаты', ['54.605754, 26.531853', '(', '*', ')']],
     ['На старых картах', ['[1]', ',', '[2]', ',', '[3]']],
     ['Списки', ['1-я мировая война']]]}]}}

In [35]:
def analyze_location_levels():
    location_levels = []
    for record in globus_places:
        place = record["place"]
        if "location" not in place:
            print(place["name"])
            continue
        location = place["location"]
        if isinstance(location, list):
            location_levels.extend(location)
        else:
            location_levels.extend([l.strip() for l in location.split(", ")])
    return set(location_levels)
analyze_location_levels()

{'Агрогородок',
 'Агрогородок (ранее деревня)',
 'Агрогородок (ранее поселок)',
 'Барановичский район',
 'Белыничский район',
 'Березинский район',
 'Берестовицкий район',
 'Берёзовский район',
 'Бешенковичский район',
 'Бобруйский район',
 'Борисовский район',
 'Брагинский район',
 'Браславский район',
 'Брестская область',
 'Брестский район',
 'Буда-Кошелевский район',
 'Бывший пригород Полоцка',
 'Быховский район',
 'Верхнедвинский район',
 'Ветковский район',
 'Вилейский район',
 'Витебская область',
 'Витебский район',
 'Волковысский район',
 'Воложинский район',
 'Вороновский район',
 'Ганцевичский район',
 'Глубокский район',
 'Глусский район',
 'Гомельская область',
 'Гомельский район',
 'Горецкий район',
 'Город',
 'Город (областной центр)',
 'Город (районный центр)',
 'Город (столица Республики Беларусь)',
 'Городокский район',
 'Городской поселок',
 'Городской поселок (районный центр)',
 'Гродненская область',
 'Гродненский район',
 'Деревня',
 'Деревня (Белицкий сельсовет)'

In [36]:
import re

def preprocess_locations():
    new_records = []
    removed = 0
    for record in globus_places:
        place = record["place"]
        location = place["location"]
        if "не существует" in location:
            removed += 1
            continue

        location_parts = [l.strip() for l in location.split(", ")]
        new_location_parts = []
        for location_part in location_parts:
            if "(" not in location_part and ")" not in location_part:
                new_location_parts.append(location_part)
                continue
            if "ныне" in location_part:
                new_location_parts.append(location_part)
                continue
            # remove part in parantheses using regex
            location_part = re.sub(r'\([^)]*\)', '', location_part).strip()
            new_location_parts.append(location_part)
        place["location"] = new_location_parts
        new_records.append(record)
    print(f"Removed {removed} records")
    return new_records

globus_places = preprocess_locations()

Removed 3 records


In [37]:
def analyze_sight_props():
    props_names = []
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            props_names.extend([name for name, _ in sight["props"]])
    return set(props_names)
analyze_sight_props()

{'Адрес',
 'Гос. цен-  ность (0..3)',
 'Гос. ценность (0..3)',
 'Дата',
 'Дата основания',
 'Дата строи-  тельства',
 'Дата строительства',
 'Дата установки',
 'Даты / обновления',
 'Даты строи-  тельства',
 'Даты строительства / обновления',
 'Координаты',
 'На старых картах',
 'Ориентир',
 'Предполагаемые  координаты',
 'Рейтинг (1..4)',
 'Рейтинг(1..4)',
 'Списки',
 'Статус',
 'Стиль'}

In [38]:
sight_props_mapping = {
    'Статус': 'status',
    'Списки': 'tags',
    'Рейтинг(1..4)': 'rating',
    'Рейтинг (1..4)': 'rating',
    'Предполагаемые  координаты': 'probable_coordinates',
    'Координаты': 'coordinates',
    'Адрес': 'address',
    'Гос. цен-  ность (0..3)': 'state_value',
    'Гос. ценность (0..3)': 'state_value',
}

def preprocess_sight_props():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "props" not in sight:
                continue

            props_dict = {}
            for prop_name, prop_value in sight["props"]:
                if prop_name in sight_props_mapping:
                    prop_name = sight_props_mapping[prop_name]
                    props_dict[prop_name] = prop_value
                else:
                    continue
            sight["props"] = props_dict
preprocess_sight_props()

In [39]:
def analyze_sight_props_values(prop_name):
    props_values = []
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            for name, value in sight["props"].items():
                if name == prop_name:
                    if isinstance(value, list):
                        props_values.extend(value)
                    else:
                        props_values.append(value)
    return set(props_values)

In [40]:
analyze_sight_props_values("status")

{'объект не сохранился'}

In [41]:
def remove_non_existing_sights():
    sights_removed = 0
    for record in globus_places:
        place = record["place"]
        filtered_sights = []
        for sight in place['sights']:
            if 'status' in sight['props'] and sight['props']['status'][0].lower() == 'объект не сохранился':
                sights_removed += 1
                continue
            if "tags" in sight["props"] and 'Утраченные недавно' in sight['props']['tags']:
                sights_removed += 1
                continue
            if sight["name"].lower() == "фотозарисовки":
                sights_removed += 1
                continue
            filtered_sights.append(sight)
        place['sights'] = filtered_sights
    print(f"Removed {sights_removed} non-existing sights")
remove_non_existing_sights()

Removed 537 non-existing sights


In [42]:
analyze_sight_props_values("tags")

{'(',
 '(?)',
 ')',
 '1-я мировая война',
 'Аптеки',
 'Арочные мосты',
 'Банки и казначейства',
 'Башни',
 'Башни оборонительного типа',
 'Борисовы камни',
 'Брамы',
 'Брамы (избранное)',
 'Бровары и винокурни',
 'Ветряные мельницы',
 'Ветряные электрогенераторы',
 'Водяные мельницы',
 'Вокзалы и ж/д станции',
 'Восстание 1830-31 гг.',
 'Восстание 1863-64 гг.',
 'Восстание Костюшко 1794 г.',
 'Гаштольды',
 'Гмины, староства, воеводства',
 'Городница',
 'Гостиницы и заезжие дворы',
 'Греко-католические церкви действующие',
 'Дворцы',
 'Деревянное зодчество (избранное)',
 'Деревянные костелы (избранное)',
 'Деревянные церкви (избранное)',
 'Дом-крепость',
 'Дома-музеи',
 'Дуга Струве',
 'Еврейское наследие',
 'Заброшенные и захороненные деревни',
 'Замки',
 'Иешивы',
 'Интересная география',
 'Интересные надмогилья',
 'Интересные названия',
 'Иоанн Павел II',
 'Кагальные колодцы',
 'Кальвинские сборы',
 'Каменные кресты',
 'Камни и валуны',
 'Каретные и гаражи',
 'Карьеры',
 'Кирхи',
 'К

In [43]:
tag_mapping = {
    '(': None,
 '(?)': None,
 ')': None,
 '1-я мировая война': '1-я мировая война',
 'Аптеки': 'Аптеки',
 'Арочные мосты': 'Арочные мосты',
 'Банки и казначейства': 'Банки и казначейства',
 'Башни': 'Башни',
 'Башни оборонительного типа': 'Башни',
 'Борисовы камни': 'Борисовы камни',
 'Брамы': 'Брамы',
 'Брамы (избранное)': 'Брамы',
 'Бровары и винокурни': 'Бровары и винокурни',
 'Ветряные мельницы': 'Ветряные мельницы',
 'Ветряные электрогенераторы': 'Ветряные электрогенераторы',
 'Водяные мельницы': 'Водяные мельницы',
 'Вокзалы и ж/д станции': 'Вокзалы и ж/д станции',
 'Восстание 1830-31 гг.': 'Восстание 1830-31 гг.',
 'Восстание 1863-64 гг.': 'Восстание 1863-64 гг.',
 'Восстание Костюшко 1794 г.': 'Восстание Костюшко 1794 г.',
 'Гаштольды': None,
 'Гмины, староства, воеводства': None,
 'Городница': 'Городница',
 'Гостиницы и заезжие дворы': 'Гостиницы и заезжие дворы',
 'Греко-католические церкви действующие': 'Греко-католические церкви',
 'Дворцы': 'Дворцы',
 'Деревянное зодчество (избранное)': 'Деревянное зодчество',
 'Деревянные костелы (избранное)': 'Деревянные костелы',
 'Деревянные церкви (избранное)': 'Деревянные церкви',
 'Дом-крепость': 'Дом-крепость',
 'Дома-музеи': 'Дом-музей',
 'Дуга Струве': 'Дуга Струве',
 'Еврейское наследие': 'Еврейское наследие',
 'Заброшенные и захороненные деревни': 'Заброшенные и захороненные деревни',
 'Замки': 'Замки',
 'Иешивы': 'Иешивы',
 'Интересная география': 'Интересная география',
 'Интересные надмогилья': 'Интересные надмогилья',
 'Интересные названия': 'Интересные названия',
 'Иоанн Павел II': 'Иоанн Павел II',
 'Кагальные колодцы': 'Кагальные колодцы',
 'Кальвинские сборы': 'Кальвинские сборы',
 'Каменные кресты': 'Каменные кресты',
 'Камни и валуны': 'Камни и валуны',
 'Каретные и гаражи': 'Каретные и гаражи',
 'Карьеры': 'Карьеры',
 'Кирхи': 'Кирха',
 'Кладбища еврейские': 'Кладбища',
 'Кладбища лютеранские': 'Кладбища',
 'Кладбища татарские': 'Кладбища',
 'Коллегиумы иезуитов': 'Коллегиумы иезуитов',
 'Колокольни (избранное)': 'Колокольни',
 'Колонны и обелиски': 'Колонны и обелиски',
 'Корпус охраны пограничья': 'Корпус охраны пограничья',
 'Корчмы и аустерии': 'Корчмы и аустерии',
 'Костелы (избранное)': 'Костелы',
 'Крепости': 'Крепости',
 'Кресты-обереги придорожные': 'Кресты-обереги придорожные',
 'Лютеранское наследие': 'Лютеранское наследие',
 'Машины-памятники': 'Машины-памятники',
 'Мельницы': 'Мельницы',
 'Мемориальные камни': 'Мемориальные камни',
 'Мечети и молельные дома': 'Мечети и молельные дома',
 'Монастыри базилиан': 'Монастыри',
 'Монастыри бенедиктинцев': 'Монастыри',
 'Монастыри бернардинцев': 'Монастыри',
 'Монастыри бонифратов': 'Монастыри',
 'Монастыри доминиканцев': 'Монастыри',
 'Монастыри иезуитов': 'Монастыри',
 'Монастыри кармелитов': 'Монастыри',
 'Монастыри картезианцев': 'Монастыри',
 'Монастыри мариан': 'Монастыри',
 'Монастыри миссионеров': 'Монастыри',
 'Монастыри назаретянок': 'Монастыри',
 'Монастыри пиаров': 'Монастыри',
 'Монастыри правосл. действ.': 'Монастыри',
 'Монастыри православные': 'Монастыри',
 'Монастыри тринитариев': 'Монастыри',
 'Монастыри францисканцев': 'Монастыри',
 'Монастыри цистерцианцев': 'Монастыри',
 'Немецкие кладбища 1-й мировой': 'Кладбища',
 'Немецкие кладбища 2-й мировой': 'Кладбища',
 'Оборонительные храмы': 'Оборонительные храмы',
 'Объекты 1500-1599 гг.': None,
 'Объекты 1600-1699 гг.': None,
 'Объекты до 1499 г.': None,
 'Озера и водохранилища': 'Озера и водохранилища',
 'Орден августинцев': 'Монастыри',
 'Орден базилиан': 'Монастыри',
 'Орден бенедиктинцев': 'Монастыри',
 'Орден бернардинцев': 'Монастыри',
 'Орден доминиканцев': 'Монастыри',
 'Орден иезуитов': 'Монастыри',
 'Орден иоаннитов (рыцарей мальтийского ордена)': 'Монастыри',
 'Орден каноников латеранских': 'Монастыри',
 'Орден каноников регулярных': 'Монастыри',
 'Орден кармелитов': 'Монастыри',
 'Орден коммунистов': 'Монастыри',
 'Орден мариавиток': 'Монастыри',
 'Орден миссионеров': 'Монастыри',
 'Орден назаретянок': 'Монастыри',
 'Орден пиаров': 'Монастыри',
 'Орден рохитов': 'Монастыри',
 'Орден трапистов': 'Монастыри',
 'Орден францисканцев': 'Монастыри',
 'Орден цистерцианцев': 'Монастыри',
 'Орден шкаплеров': 'Монастыри',
 'Отечественная война 1812 г.': 'Отечественная война 1812 г.',
 'Паровые мельницы': 'Мельницы',
 'Паромы': 'Паромы',
 'Пещеры':  'Пещеры',
 'Плебании': 'Плебании',
 'Подвесные мосты': 'Подвесные мосты',
 'Пожарные': 'Пожарные',
 'Польские кладбища войны 1919-21 гг.': 'Кладбища',
 'Почтовые станции и заставные дома': 'Почтовые станции и заставные дома',
 'Примечательные могилы': 'Могилы',
 'Производство': 'Производство',
 'Промышленность': 'Промышленность',
 'Протестантские храмы действующие': 'Протестантские храмы',
 'Радзивиллы': 'Радзивиллы',
 'Ратуши': 'Ратуши',
 'Реки и каналы': 'Реки и каналы',
 'Русские кладбища 1-й мировой': 'Кладбища',
 'Русско-французская война 1812 г.': 'Русско-французская война 1812 г.',
 'Сапеги': 'Сапеги',
 'Северная война 1700-21 гг.': 'Северная война 1700-21 гг.',
 'Синагога': 'Синагоги',
 'Синагоги':  'Синагоги',
 'Синагоги (избранное)': 'Синагоги',
 'Синагоги действующие': 'Синагоги',
 'Солнечные часы': 'Солнечные часы',
 'Сталин': None,
 'Старообрядческие церкви': 'Старообрядческие церкви',
 'Старые дороги': 'Старые дороги',
 'Татарское наследие': 'Татарское наследие',
 'Топ 100': None,
 'Топ 20': None,
 'Торговые ряды': 'Торговые ряды',
 'Тюрьмы': 'Тюрьмы',
 'Усадьбы и фольварки': 'Усадьбы и фольварки',
 'Фотозарисовки': None,
 'Ходкевичи': 'Ходкевичи',
 'Холодная война': 'Холодная война',
 'Хрептовичи': 'Хрептовичи',
 'Церкви (избранное)': 'Церкви',
 'Часовни (избранное)': 'Часовни',
 'Часовни придорожные (избранное)': 'Часовни',
 'Шуховские башни': 'Башни',
 'Языческое наследие': 'Языческое наследие',
 'августинцев': None
 }

def preprocess_tags():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "tags" not in sight["props"]:
                continue
            tags = sight["props"]["tags"]
            new_tags = []
            for tag in tags:
                if tag in tag_mapping:
                    tag = tag_mapping[tag]
                    if tag is not None:
                        new_tags.append(tag)
                else:
                    print(f"Unknown tag: {tag}")
            sight["props"]["tags"] = new_tags
preprocess_tags()

In [44]:
analyze_sight_props_values("tags")

{'1-я мировая война',
 'Аптеки',
 'Арочные мосты',
 'Банки и казначейства',
 'Башни',
 'Борисовы камни',
 'Брамы',
 'Бровары и винокурни',
 'Ветряные мельницы',
 'Ветряные электрогенераторы',
 'Водяные мельницы',
 'Вокзалы и ж/д станции',
 'Восстание 1830-31 гг.',
 'Восстание 1863-64 гг.',
 'Восстание Костюшко 1794 г.',
 'Городница',
 'Гостиницы и заезжие дворы',
 'Греко-католические церкви',
 'Дворцы',
 'Деревянное зодчество',
 'Деревянные костелы',
 'Деревянные церкви',
 'Дом-крепость',
 'Дом-музей',
 'Дуга Струве',
 'Еврейское наследие',
 'Заброшенные и захороненные деревни',
 'Замки',
 'Иешивы',
 'Интересная география',
 'Интересные надмогилья',
 'Интересные названия',
 'Иоанн Павел II',
 'Кагальные колодцы',
 'Кальвинские сборы',
 'Каменные кресты',
 'Камни и валуны',
 'Каретные и гаражи',
 'Карьеры',
 'Кирха',
 'Кладбища',
 'Коллегиумы иезуитов',
 'Колокольни',
 'Колонны и обелиски',
 'Корпус охраны пограничья',
 'Корчмы и аустерии',
 'Костелы',
 'Крепости',
 'Кресты-обереги прид

In [45]:
analyze_sight_props_values("rating")

{'взглянуть', 'увидеть интересно', 'увидеть обязательно', 'увидеть по дороге'}

In [46]:
rating_mapping = {
    'взглянуть': 0, 'увидеть интересно': 2, 'увидеть обязательно': 3, 'увидеть по дороге': 1
}

def preprocess_rating():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "rating" not in sight["props"]:
                continue
            rating = sight["props"]["rating"][0]
            if rating in rating_mapping:
                sight["props"]["rating"] = rating_mapping[rating]
            else:
                print(f"Unknown rating: {rating}")
preprocess_rating()
analyze_sight_props_values("rating")

{0, 1, 2, 3}

In [47]:
analyze_sight_props_values('state_value')

{'0', '1', '2', '3'}

In [48]:
def preprocess_state_value():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "state_value" not in sight["props"]:
                continue
            state_value = sight["props"]["state_value"][0]
            sight["props"]["state_value"] = 3 - int(state_value)
preprocess_state_value()
analyze_sight_props_values('state_value')

{0, 1, 2, 3}

In [49]:
analyze_sight_props_values('address')

{'1-',
 '17-',
 '2-',
 '23-',
 '3-',
 '50-',
 'Пролетарская, 51',
 'го Интернационала, 28',
 'го Интернационала, 49',
 'го Интернационала, 50',
 'го Мая',
 'го Мая, 17',
 'го Мая, дом 11',
 'го Мая, дом 9',
 'го Сентября',
 'го Сентября, 11',
 'го Сентября, 28',
 'й пер. Крупской, 8',
 'летия ВЛКСМ, 18',
 'пер. Банный, 1',
 'пер. Больничный, 13',
 'пер. Ильюшенко, 14',
 'пер. Ломоносова, 7/1',
 'пер. Музейный, 12',
 'пер. Почтовый, 3',
 'пер. Титова',
 'пер. Факельный',
 'перекресток улиц Гагарина, Матросова, Я. Коласа',
 'пл. Комсомольская, 1',
 'пл. Красная, 5',
 'пл. Ленина, 1',
 'пл. Ленина, 11',
 'пл. Ленина, 13',
 'пл. Ленина, 15',
 'пл. Ленина, 5',
 'пл. Ленина, 6',
 'пл. Ленина, 7, 9 и 11',
 'пл. Ленина, 9',
 'пл. Льва Сапеги',
 'пл. Привокзальная, 3',
 'пл. Свободы',
 'пл. Свободы, 6',
 'пл. Советская',
 'пл. Тызенгауза, 2/2',
 'пр. Луначарского, 8',
 'пр. Машерова, 28',
 'пр. Независимости, 15',
 'пр. Революции',
 'пр. Фрунзе, 11"А"',
 'угол ул. Оршанской и ул. Дружбы, на вос

In [50]:
def preprocess_address():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "address" not in sight["props"]:
                continue
            address = " ".join(sight["props"]["address"])
            # remove space after -
            address = address.replace("- ", "-").replace('"', '')
            sight["props"]["address"] = address
preprocess_address()
analyze_sight_props_values('address')

{'1-й пер. Крупской, 8',
 'Пролетарская, 51',
 'пер. Банный, 1',
 'пер. Больничный, 13',
 'пер. Ильюшенко, 14',
 'пер. Ломоносова, 7/1',
 'пер. Музейный, 12',
 'пер. Почтовый, 3',
 'пер. Титова',
 'пер. Факельный',
 'перекресток улиц Гагарина, Матросова, Я. Коласа',
 'пл. Комсомольская, 1',
 'пл. Красная, 5',
 'пл. Ленина, 1',
 'пл. Ленина, 11',
 'пл. Ленина, 13',
 'пл. Ленина, 15',
 'пл. Ленина, 5',
 'пл. Ленина, 6',
 'пл. Ленина, 7, 9 и 11',
 'пл. Ленина, 9',
 'пл. Льва Сапеги',
 'пл. Привокзальная, 3',
 'пл. Свободы',
 'пл. Свободы, 6',
 'пл. Советская',
 'пл. Тызенгауза, 2/2',
 'пр. Луначарского, 8',
 'пр. Машерова, 28',
 'пр. Независимости, 15',
 'пр. Революции',
 'пр. Фрунзе, 11А',
 'угол ул. Оршанской и ул. Дружбы, на восточной окраине Гомеля (бывшая дер. Мильча)',
 'угол улиц Кирова и Карповича',
 'ул. 1-го Мая',
 'ул. 1-го Мая, 17',
 'ул. 1-го Мая, дом 11',
 'ул. 1-го Мая, дом 9',
 'ул. 17-го Сентября',
 'ул. 17-го Сентября, 11',
 'ул. 17-го Сентября, 28',
 'ул. 2-я Заозерная'

In [51]:
def preprocess_sights_coordinates():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "coordinates" not in sight["props"]:
                continue
            if not isinstance(sight["props"]["coordinates"], list):
                continue
            coordinates = sight["props"]["coordinates"][0]
            sight["props"]["coordinates"] = tuple(float(c.strip()) for c in coordinates.split(","))
preprocess_sights_coordinates()

In [52]:
def preprocess_sights_probable_coordinates():
    for record in globus_places:
        place = record["place"]
        for sight in place["sights"]:
            if "probable_coordinates" not in sight["props"]:
                continue
            if not isinstance(sight["props"]["probable_coordinates"], list):
                continue
            coordinates = sight["props"]["probable_coordinates"][0]
            sight["props"]["probable_coordinates"] = tuple(float(c.strip()) for c in coordinates.split(","))
preprocess_sights_probable_coordinates()

In [53]:
def preprocess_globus_data():
    processed_data = []
    for record in globus_places:
        place = record["place"]
        location = list(reversed(place['location']))
        place_type = None
        if len(location) == 3:
            place_type = location[-1]
            location = tuple(location[:-1])
        elif len(location) == 2:
            location = tuple(location)
        else:
            print(location)
            continue

        if place['name'] == 'Минск':
            location = ('Минская область',)

        for sight in place["sights"]:
            if "props" in sight:
                for prop_name, prop_value in sight["props"].items():
                    sight[prop_name] = prop_value
                del sight["props"]
            sight["name"] = sight["name"].replace("  ", " ")
            if sight['image'] is not None and 'https://globustut.by/' in sight['image']:
                sight['image'] = sight['image'].replace('index.htm', '')
        if "coords" not in place or not place["coords"]:
            place["coords"] = next((sight["coordinates"] for sight in place["sights"] if "coordinates" in sight), None)
        place["url"] = {'href': record["url"], 'text': place['name']}
        place["rating"] = 4 - record["rating"]
        place['location'] = location
        place['type'] = place_type
        processed_data.append(place)
    return processed_data
processed_globus_data = preprocess_globus_data()
processed_globus_data[0]

{'name': 'Абрамовщина 3-я',
 'location': ('Гродненская область', 'Сморгонский район'),
 'coords': [54.60153, 26.53344],
 'sights': [{'name': 'оборонительные сооружения 1-й мировой войны',
   'image': 'https://orda.of.by/.ga/a/abramovsch/oboron_main/nf/abramovschina_3-ya-kuhnya_1mir-06.05.06-d-179.jpg',
   'rating': 1,
   'coordinates': (54.605754, 26.531853),
   'tags': ['1-я мировая война']}],
 'url': {'href': 'https://globustut.by/abramovsch/index.htm',
  'text': 'Абрамовщина 3-я'},
 'rating': 1,
 'type': None}

In [54]:
import json

with open("globus_processed.json", "w", encoding="utf-8") as f:
    json.dump(processed_globus_data, f, ensure_ascii=False, indent=2)